In [ ]:
from scipy.signal import chirp, sweep_poly
import numpy as np
import matplotlib.pyplot as plt
t = np.linspace(0, 1, 720)
container_sim = 20+np.ceil(20*chirp(t, f0=1, f1=15, t1=1, method='linear',phi=179))
np.random.seed(seed=1)
uzer = np.random.randint(low=1,high=150,size=720)
plt.figure(figsize=(10,5), dpi=300)
plt.plot(container_sim)
plt.show()
plt.figure(figsize=(10,5), dpi=300)
plt.plot(uzer)
plt.show()


In [ ]:
#### Importing Necessary Libraries
import subprocess
import time
import sys
import time
import math
from kubernetes import client, config
import tensorflow as tf
import tensorflow.contrib.layers as layers
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/jupyter/google.json"
output = subprocess.check_output('kubectl get pods', shell=True)
tf.reset_default_graph()  # This is to reset the graph

############## 
config.load_kube_config()
DEPLOYMENT_NAME = 'wordpress'
DEPLOYMENT_NS = 'default'


api_instance = client.ExtensionsV1beta1Api()

def get_replica_and_ready():
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    return api_response.status.replicas, api_response.status.ready_replicas

def set_replica_num(rnum):
    rnum = int(rnum)
    if rnum < 1:
        rnum = 1
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    api_response.spec.replicas = rnum
    api_instance.patch_namespaced_deployment_scale(DEPLOYMENT_NAME, DEPLOYMENT_NS, api_response)
    
print(get_replica_and_ready())
u_pre = 2
set_replica_num(u_pre)



In [ ]:
#%% Change working directory from the workspace root to the ipynb file location. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'examples'))
    print(os.getcwd())
except:
    pass
import ddsl_load_tester as load_tester


#%%
def custom_sensing():
    r1,r2 = get_replica_and_ready()
    return {'r1':r1 , 'r2':r2}
lt = load_tester.DdslLoadTester(hatch_rate=1, temp_stat_max_len=5, base='http://35.232.158.172:8089/')

lt.custom_sensing = custom_sensing
# Testing the function
lt.custom_sensing()
from tqdm.auto import tqdm
tqdm.pandas()

loop_timer = load_tester.TimerClass()
total_timer = load_tester.TimerClass()
user_sequence = [1,10,20,30,40,50,60,70,80,90,100]
container_Request = container_sim
lt.custom_sensing = custom_sensing
lt.change_count(user_sequence[0])
lt.start_capturing()

loop_time_in_secs = load_tester.get_loop_time_in_secs('20s')

loop_timer.tic()
total_timer.tic()
results = None

for j in container_Request :
    for i in tqdm(range(len(container_sim))):
        user_count = j
        lt.change_count(user_count)

        sleep_time = loop_time_in_secs - loop_timer.toc()
        if sleep_time > 0:
            time.sleep(sleep_time)

        loop_timer.tic()

        result = lt.get_all_stats()
        df_result = pd.DataFrame(data=result)

        if results is None:
            results = df_result
        else:
            results = results.append(df_result)



        #####Identification
        if (result['current_response_time_average'])!=[]:
            if result['current_response_time_average'][-1]!=None :
                u_pre = container_sim[i]
                if np.isnan(u_pre):
                    u_pre = default
                if u_pre >= 40 :
                    u_pre = 40
                try:
                    set_replica_num(u_pre)
                except:
                    print("Error occured !!!!!")
            print("iteration : ",i ,"average RSPT is: ",result['current_response_time_average'][-1],"Number of Replica Ready: ",result['custom_r1'][-1], "Number of Users: ",user_count)
lt.stop_test()

results, filename = lt.prepare_results_from_df(results)

results.head()

In [ ]:
res = results
res = res[res['total_rps'] > 0]
responsetime_avg = np.average(res['current_response_time_average'])
responsetime_var = np.sqrt(np.var(res['current_response_time_average']))
container_avg = np.average(res['custom_r2'])
container_var = np.sqrt(np.var(res['custom_r2']))
print("average res : ", responsetime_avg , "Var Res : ",responsetime_var, "average Container : ",container_avg, "Var Container : ", container_var )
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')

plt.figure(figsize=(8,18))
plt.subplot(511)
#plt.plot(res['elapsed_min'], res['min_response_time'], label='min_response_time')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_50'], label='median_response_time')
plt.plot(res['elapsed_min'], res['current_response_time_average'], label='avg_response_time')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_95'], label='95th percentile')
#plt.plot(res['elapsed_min'], res['max_response_time'], label='max_response_time')

plt.xlabel('Time (minutes)')
plt.ylabel('Average Response Time (ms)')
plt.legend()

plt.subplot(512)
plt.plot(res['elapsed_min'], res['user_count'])
plt.xlabel('Time (minutes)')
plt.ylabel('Num of Users')

plt.subplot(513)
plt.plot(res['elapsed_min'], res['total_rps'])
plt.xlabel('Time (minutes)')
plt.ylabel('Throughput (req/s)')

plt.subplot(514)
plt.plot(res['elapsed_min'], res['fail_ratio'])
plt.xlabel('Time (minutes)')
plt.ylabel('Fail Ratio')

plt.subplot(515)
plt.plot(res['elapsed_min'], res['custom_r1'])
plt.plot(res['elapsed_min'], res['custom_r2'])
plt.xlabel('Time (minutes)')
plt.ylabel('Custom Value')



filename = filename.replace('aa.csv', '')
plt.savefig(filename + '.png', dpi=300)
plt.savefig(filename + '.pdf')
plt.show()


lt.stop_test()

results, filename = lt.prepare_results_from_df(results)

results.head()